In [1]:
import pandas as pd
from collections import defaultdict
import warnings
import visualize
import neat
import multiprocessing
import os
import matplotlib.pyplot as plt

import numpy as np

In [2]:
necoc = 500
generations = int(3000/necoc)
generations

6

In [3]:
def softmax(x, axis=1):
    # 计算每行的最大值
    row_max = x.max(axis=axis)
 
    # 每行元素都需要减去对应的最大值，否则求exp(x)会溢出，导致inf情况
    row_max=row_max.reshape(-1, 1)
    x = x - row_max
 
    # 计算e的指数次幂
    x_exp = np.exp(x)
    x_sum = np.sum(x_exp, axis=axis, keepdims=True)
    s = x_exp / x_sum
    return s

In [4]:
df = pd.read_csv('letter-recognition.data',header=None)

train_df = df[:1600]
test_df = df[16000:16000+400]

X = train_df.ix[:,1:16].to_numpy() 
y = np.array([ord(l)-ord('A') for l in train_df.ix[:,0].to_numpy()])

testX = test_df.ix[:,1:16].to_numpy() 
testy = np.array([ord(l)-ord('A') for l in test_df.ix[:,0].to_numpy()])

In [5]:
def get_winner(class1):
    
    numbers = [*range(26)] # number 2 > number 1
    number_of_classification = len(numbers)
    #number_of_sampling = 100

    list_y = y.tolist()
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

        
    class2 = []

    class1_indexes = []
    class2_indexes = []

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)

    print("number of class1:{}".format(len(class1_label)))
    print("number of class2:{}".format(len(class2_label)))

    samplesize = 1600
    x_inputs =[tuple(c) for c in X[class1_indexes].tolist()] + [tuple(c) for c in X[class2_indexes].tolist()]
    x_outputs = [tuple([c]) for c in class1_label + class2_label]
    
    
    def eval_genomes(genomes, config):
        for genome_id, genome in genomes:
            net = neat.nn.FeedForwardNetwork.create(genome, config)

            outputs = []
            for xi in x_inputs:
                output = net.activate(xi)
                outputs.append(output)


            px_outputs = softmax(np.array(outputs).reshape(samplesize, 2), axis=1)
            # the index of maximum in each line
            pred_outputs = np.argmax(px_outputs, axis = 1)
            real_outputs = np.array(x_outputs).reshape(samplesize,)

            acc = np.sum(pred_outputs == real_outputs)/samplesize

            genome.fitness = acc

    def run(config_file):
        # Load configuration.
        config = neat.Config(
            neat.DefaultGenome,
            neat.DefaultReproduction,
            neat.DefaultSpeciesSet,
            neat.DefaultStagnation,
            config_file,
        )

        # Create the population, which is the top-level object for a NEAT run.
        p = neat.Population(config)

        # add a stdout reporter to show progress in the terminal
        reporter = neat.StdOutReporter(False)
        p.add_reporter(reporter)
        stats = neat.StatisticsReporter()
        p.add_reporter(stats)
        #checkpointer = neat.Checkpointer(100)
        #p.add_reporter(checkpointer)
        # Run for up to 300 generations.
        winner = p.run(eval_genomes, generations)

        return [stats, winner]
    
    local_dir = os.getcwd()
    config_path = os.path.join(local_dir, "config-feedforward-letter2")
    [stats, winner] = run(config_path)
    
    return winner

In [6]:
import numpy as np
import random

number_codes = []
for i in range(2**(26-1), 2**26-1):
    if (i % 1000000) == 0:
            print(i)
    for j in bin(i).split('b')[1]:
        number_codes.append(int(j))
        


number_codes = np.array(number_codes).reshape(2**(26 - 1) - 1,26).T     

slice = random.sample(range(2**(26 - 1) - 1), necoc)

number_codes = number_codes[:,slice].tolist()

classes = []
for i in range(necoc):
    code_list = np.array(number_codes).T[i]
    classes.append(np.where(code_list==1)[0].tolist())

matrix_errors = []

for i in range(25):
    for j in range(i+1,26):
        matrix_errors.append(necoc - np.sum(np.array(number_codes[i]) == np.array(number_codes[j])))
min_error = np.min(matrix_errors)

print(min_error)

34000000
35000000
36000000
37000000
38000000
39000000
40000000
41000000
42000000
43000000
44000000
45000000
46000000
47000000
48000000
49000000
50000000
51000000
52000000
53000000
54000000
55000000
56000000
57000000
58000000
59000000
60000000
61000000
62000000
63000000
64000000
65000000
66000000
67000000
217


In [7]:
winner_list = []

for i,class1 in enumerate(classes):
    print("running:{}/30".format(i))
    winner = get_winner(class1)
    winner_list.append(winner)

running:0/30
number of class1:973
number of class2:627

 ****** Running generation 0 ****** 

Population's average fitness: 0.48988 stdev: 0.09908
Best fitness: 0.63250 - size: (2, 3) - species 4 - id 131
Average adjusted fitness: 0.105
Mean genetic distance 2.646, standard deviation 0.491
Population of 199 members in 12 species
Total extinctions: 0
Generation time: 2.852 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.52962 stdev: 0.09625
Best fitness: 0.63375 - size: (2, 2) - species 2 - id 380
Average adjusted fitness: 0.154
Mean genetic distance 2.770, standard deviation 0.531
Population of 200 members in 17 species
Total extinctions: 0
Generation time: 3.110 sec (2.981 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.54411 stdev: 0.09434
Best fitness: 0.65625 - size: (4, 7) - species 3 - id 394
Average adjusted fitness: 0.179
Mean genetic distance 2.848, standard deviation 0.539
Population of 200 members in 19 species
Total

Population's average fitness: 0.50258 stdev: 0.02351
Best fitness: 0.58437 - size: (2, 3) - species 2 - id 9
Average adjusted fitness: 0.068
Mean genetic distance 2.679, standard deviation 0.501
Population of 201 members in 10 species
Total extinctions: 0
Generation time: 2.690 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.50787 stdev: 0.02406
Best fitness: 0.58437 - size: (2, 3) - species 2 - id 9
Average adjusted fitness: 0.064
Mean genetic distance 2.770, standard deviation 0.544
Population of 200 members in 14 species
Total extinctions: 0
Generation time: 3.070 sec (2.880 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.51000 stdev: 0.02719
Best fitness: 0.58437 - size: (2, 3) - species 4 - id 9
Average adjusted fitness: 0.092
Mean genetic distance 2.727, standard deviation 0.544
Population of 199 members in 14 species
Total extinctions: 0
Generation time: 4.605 sec (3.455 average)

 ****** Running generation 3 ****** 

Po

Population's average fitness: 0.49983 stdev: 0.02078
Best fitness: 0.58875 - size: (2, 3) - species 6 - id 118
Average adjusted fitness: 0.081
Mean genetic distance 2.781, standard deviation 0.538
Population of 199 members in 17 species
Total extinctions: 0
Generation time: 3.232 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.50703 stdev: 0.02269
Best fitness: 0.58875 - size: (2, 3) - species 6 - id 118
Average adjusted fitness: 0.084
Mean genetic distance 2.871, standard deviation 0.565
Population of 200 members in 18 species
Total extinctions: 0
Generation time: 4.544 sec (3.888 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.50738 stdev: 0.02222
Best fitness: 0.59000 - size: (3, 6) - species 1 - id 464
Average adjusted fitness: 0.063
Mean genetic distance 2.914, standard deviation 0.566
Population of 201 members in 20 species
Total extinctions: 0
Generation time: 4.399 sec (4.058 average)

 ****** Running generation 3 *****

Population's average fitness: 0.50737 stdev: 0.10466
Best fitness: 0.63438 - size: (2, 3) - species 6 - id 189
Average adjusted fitness: 0.145
Mean genetic distance 2.756, standard deviation 0.595
Population of 201 members in 14 species
Total extinctions: 0
Generation time: 3.379 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.57012 stdev: 0.09002
Best fitness: 0.63438 - size: (2, 3) - species 11 - id 189
Average adjusted fitness: 0.173
Mean genetic distance 2.735, standard deviation 0.602
Population of 203 members in 14 species
Total extinctions: 0
Generation time: 3.956 sec (3.668 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.56421 stdev: 0.09458
Best fitness: 0.65500 - size: (4, 6) - species 3 - id 536
Average adjusted fitness: 0.185
Mean genetic distance 2.682, standard deviation 0.552
Population of 200 members in 14 species
Total extinctions: 0
Generation time: 5.978 sec (4.438 average)

 ****** Running generation 3 ****

Population's average fitness: 0.49403 stdev: 0.09092
Best fitness: 0.66687 - size: (2, 3) - species 8 - id 136
Average adjusted fitness: 0.170
Mean genetic distance 2.807, standard deviation 0.555
Population of 200 members in 15 species
Total extinctions: 0
Generation time: 4.272 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.52617 stdev: 0.09007
Best fitness: 0.70250 - size: (3, 4) - species 7 - id 315
Average adjusted fitness: 0.160
Mean genetic distance 2.814, standard deviation 0.529
Population of 199 members in 18 species
Total extinctions: 0
Generation time: 5.601 sec (4.936 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.53732 stdev: 0.08987
Best fitness: 0.70250 - size: (3, 4) - species 6 - id 315
Average adjusted fitness: 0.198
Mean genetic distance 2.892, standard deviation 0.537
Population of 200 members in 22 species
Total extinctions: 0
Generation time: 3.432 sec (4.435 average)

 ****** Running generation 3 *****

...

Population's average fitness: 0.50827 stdev: 0.06681
Best fitness: 0.63062 - size: (2, 3) - species 3 - id 9
Average adjusted fitness: 0.159
Mean genetic distance 2.553, standard deviation 0.489
Population of 201 members in 8 species
Total extinctions: 0
Generation time: 2.629 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.52201 stdev: 0.06596
Best fitness: 0.63250 - size: (2, 3) - species 3 - id 373
Average adjusted fitness: 0.138
Mean genetic distance 2.630, standard deviation 0.508
Population of 201 members in 11 species
Total extinctions: 0
Generation time: 3.153 sec (2.891 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.53537 stdev: 0.06416
Best fitness: 0.63500 - size: (3, 5) - species 6 - id 548
Average adjusted fitness: 0.147
Mean genetic distance 2.749, standard deviation 0.505
Population of 199 members in 15 species
Total extinctions: 0
Generation time: 3.061 sec (2.948 average)

 ****** Running generation 3 ****** 


Population's average fitness: 0.51718 stdev: 0.19602
Best fitness: 0.72813 - size: (2, 3) - species 3 - id 27
Average adjusted fitness: 0.233
Mean genetic distance 2.784, standard deviation 0.560
Population of 199 members in 17 species
Total extinctions: 0
Generation time: 2.702 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.59099 stdev: 0.18080
Best fitness: 0.73125 - size: (2, 3) - species 1 - id 267
Average adjusted fitness: 0.315
Mean genetic distance 2.901, standard deviation 0.600
Population of 202 members in 19 species
Total extinctions: 0
Generation time: 3.078 sec (2.890 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.63832 stdev: 0.15079
Best fitness: 0.73125 - size: (2, 3) - species 12 - id 267
Average adjusted fitness: 0.361
Mean genetic distance 2.906, standard deviation 0.608
Population of 202 members in 19 species
Total extinctions: 0
Generation time: 3.162 sec (2.981 average)

 ****** Running generation 3 *****

Population's average fitness: 0.49842 stdev: 0.02336
Best fitness: 0.57625 - size: (2, 3) - species 9 - id 124
Average adjusted fitness: 0.082
Mean genetic distance 2.704, standard deviation 0.492
Population of 199 members in 16 species
Total extinctions: 0
Generation time: 2.723 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.50861 stdev: 0.02377
Best fitness: 0.59000 - size: (3, 5) - species 3 - id 276
Average adjusted fitness: 0.072
Mean genetic distance 2.811, standard deviation 0.546
Population of 199 members in 23 species
Total extinctions: 0
Generation time: 3.019 sec (2.871 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.50981 stdev: 0.02881
Best fitness: 0.59000 - size: (3, 5) - species 7 - id 276
Average adjusted fitness: 0.089
Mean genetic distance 2.881, standard deviation 0.548
Population of 200 members in 27 species
Total extinctions: 0
Generation time: 3.441 sec (3.061 average)

 ****** Running generation 3 *****

Population's average fitness: 0.50040 stdev: 0.03146
Best fitness: 0.61062 - size: (2, 3) - species 2 - id 83
Average adjusted fitness: 0.090
Mean genetic distance 2.705, standard deviation 0.538
Population of 199 members in 13 species
Total extinctions: 0
Generation time: 2.784 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.51306 stdev: 0.03557
Best fitness: 0.64563 - size: (3, 3) - species 1 - id 297
Average adjusted fitness: 0.182
Mean genetic distance 2.700, standard deviation 0.534
Population of 200 members in 13 species
Total extinctions: 0
Generation time: 3.172 sec (2.978 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.51600 stdev: 0.04171
Best fitness: 0.64938 - size: (4, 6) - species 1 - id 544
Average adjusted fitness: 0.143
Mean genetic distance 2.675, standard deviation 0.582
Population of 199 members in 13 species
Total extinctions: 0
Generation time: 3.292 sec (3.083 average)

 ****** Running generation 3 ******

Population's average fitness: 0.47932 stdev: 0.16835
Best fitness: 0.70500 - size: (2, 3) - species 5 - id 76
Average adjusted fitness: 0.186
Mean genetic distance 2.678, standard deviation 0.464
Population of 200 members in 14 species
Total extinctions: 0
Generation time: 2.821 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.57991 stdev: 0.15713
Best fitness: 0.71000 - size: (3, 5) - species 4 - id 349
Average adjusted fitness: 0.271
Mean genetic distance 2.696, standard deviation 0.481
Population of 201 members in 15 species
Total extinctions: 0
Generation time: 3.021 sec (2.921 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.61696 stdev: 0.14393
Best fitness: 0.71000 - size: (3, 5) - species 4 - id 349
Average adjusted fitness: 0.310
Mean genetic distance 2.686, standard deviation 0.489
Population of 201 members in 18 species
Total extinctions: 0
Generation time: 3.384 sec (3.076 average)

 ****** Running generation 3 ******

In [8]:
local_dir = os.getcwd()
config_path = os.path.join(local_dir, "config-feedforward-letter2")

config = neat.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    config_path,
)

def get_pred_real(i, j):
    winner_net = neat.nn.FeedForwardNetwork.create(winner_list[i], config)
    
    numbers = [*range(26)]
    class1 = classes[i]
    class2 = []
    class1_indexes = []
    class2_indexes = []
    
    test_X = testX[j]
    test_y = testy[j]

    list_y = [test_y]
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)


    testsamplesize = 1
    test_x_inputs =[tuple(test_X)]
    test_x_outputs = [tuple([c]) for c in class1_label + class2_label]

    outputs = []
    for xi in test_x_inputs:
        output = winner_net.activate(xi)
        outputs.append(output)

    px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 2), axis=1)
    # the index of maximum in each line
    pred_outputs = np.argmax(px_outputs, axis = 1)
    real_outputs = np.array(test_x_outputs).reshape(testsamplesize,)

    
    return [pred_outputs, real_outputs]

from random import randint
pred_value = []
error_list = []

for j in range(400):
    pred = []
    for i in range(necoc):
        [pred_outputs, real_outputs] = get_pred_real(i, j)
        #print(pred_outputs, real_outputs)
        pred.append(pred_outputs)
    #print(np.array(pred).T)
    
    error = []
    for i in range(26):
        error.append(necoc - np.sum(number_codes[i] == np.array(pred).T))
    #print(error)
    
    pred_value.append(np.where(error==np.min(error)))
    error_list.append(np.min(error))
    #print(np.where(error==np.min(error)) )
list_P = []
for i in pred_value:
    if (len(i[0])) == 1:
        list_P.append(i[0][0])
    else:
        random_pick = randint(0, len(i[0])-1)
        list_P.append(i[0][random_pick])
np.sum(list_P == testy)/400

0.495

In [9]:
def get_pred_train(i, j):
    winner_net = neat.nn.FeedForwardNetwork.create(winner_list[i], config)
    
    numbers = [*range(26)]
    class1 = classes[i]
    class2 = []
    class1_indexes = []
    class2_indexes = []
    
    test_X = X[j]
    test_y = y[j]

    list_y = [test_y]
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)


    testsamplesize = 1
    test_x_inputs =[tuple(test_X)]
    test_x_outputs = [tuple([c]) for c in class1_label + class2_label]

    outputs = []
    for xi in test_x_inputs:
        output = winner_net.activate(xi)
        outputs.append(output)

    px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 2), axis=1)
    # the index of maximum in each line
    pred_outputs = np.argmax(px_outputs, axis = 1)
    real_outputs = np.array(test_x_outputs).reshape(testsamplesize,)

    
    return [pred_outputs, real_outputs]

pred_value = []
error_list = []

for j in range(1600):
    pred = []
    for i in range(necoc):
        [pred_outputs, real_outputs] = get_pred_train(i, j)
        #print(pred_outputs, real_outputs)
        pred.append(pred_outputs)
    #print(np.array(pred).T)
    
    error = []
    for i in range(26):
        error.append(26 - np.sum(number_codes[i] == np.array(pred).T))
    #print(error)
    
    pred_value.append(np.where(error==np.min(error)))
    error_list.append(np.min(error))
    #print(np.where(error==np.min(error)) )

list_P = []
for i in pred_value:
    if (len(i[0])) == 1:
        list_P.append(i[0][0])
    else:
        random_pick = randint(0, len(i[0])-1)
        list_P.append(i[0][random_pick])
print("Trainset acc:{}".format(np.sum(list_P == y)/1600))
winner_fitness = []
for winner in winner_list:
    #print(winner.fitness)
    winner_fitness.append(winner.fitness)
print("Avg Base Learner:{}".format(np.mean(winner_fitness)))

Trainset acc:0.579375
Avg Base Learner:0.6566212499999999


In [10]:
# program to check if there is exist a path between two vertices 
# of a graph 

from collections import defaultdict 

#This class represents a directed graph using adjacency list representation 
class Graph: 

	def __init__(self,vertices): 
		self.V= vertices #No. of vertices 
		self.graph = defaultdict(list) # default dictionary to store graph 

	# function to add an edge to graph 
	def addEdge(self,u,v): 
		self.graph[u].append(v) 
	
	# Use BFS to check path between s and d 
	def isReachable(self, s, d): 
		# Mark all the vertices as not visited 
		visited =[False]*(self.V) 

		# Create a queue for BFS 
		queue=[] 

		# Mark the source node as visited and enqueue it 
		queue.append(s) 
		visited[s] = True

		while queue: 

			#Dequeue a vertex from queue 
			n = queue.pop(0) 
			
			# If this adjacent node is the destination node, 
			# then return true 
			if n == d: 
				return True

			# Else, continue to do BFS 
			for i in self.graph[n]: 
				if visited[i] == False: 
					queue.append(i) 
					visited[i] = True
		# If BFS is complete without visited d 
		return False
    
    
def findAllPath(graph,start,end,path=[]):
    path = path +[start]
    if start == end:
        return [path]
 
    paths = [] #存储所有路径    
    for node in graph[start]:
        if node not in path:
            newpaths = findAllPath(graph,node,end,path) 
            for newpath in newpaths:
                paths.append(newpath)
    return paths

numbers = [*range(2)]
number_of_classification = len(numbers)

list_nodes_number = []
list_connection_number = []

for winner in winner_list:
    used_nodes = list(winner.nodes.keys())
    # create a dict for mapping 
    l1 = numbers + config.genome_config.input_keys + used_nodes[number_of_classification:]
    l2 = range(len(l1))
    dict_nodes = dict(zip(l1, l2))

    ### 将节点加入图
    g = Graph(len(l1))# inputs + outputs + used  
    # add connections in the graph
    for cg in winner.connections.values():
        if cg.enabled:
            g.addEdge(dict_nodes[cg.key[0]],dict_nodes[cg.key[1]])

    v = numbers
    list_connections = []
    for vi in v:
        for u in range(number_of_classification, len(config.genome_config.input_keys) + number_of_classification):
            list_connections.append(g.isReachable(u, vi))

    nodes_inputs = range(number_of_classification, number_of_classification+ 64)
    nodes_outputs = range(number_of_classification)
    nodes_mid = range(number_of_classification + 64, len(dict_nodes))

    # 找到所有输入和输出连接的路
    all_path = []
    for u in nodes_inputs:
        for v in nodes_outputs:
            path = findAllPath(g.graph, u, v)
            if path:
                all_path = all_path + path

    # 得到最长路，以及每个节点在哪个层级
    max_length = max([len(x) for x in all_path])
    nodes_tuples_list = []
    for path in all_path:
        #print(path)
        for node in path:
            nodes_tuples_list.append([node, path.index(node)])

    # 确定节点的层级（消除重复）       
    nodes_tuples_fixed_list = []
    for index in range(max_length):
        for nodes in nodes_tuples_list:
            node, node_index = nodes[0], nodes[1]
            if node_index == index: 
                if nodes not in nodes_tuples_fixed_list:
                    nodes_tuples_fixed_list.append(nodes)


    # 所有有连接的节点
    all_used_nodes = []
    for node in nodes_tuples_fixed_list:
        if node[0] not in all_used_nodes:
            all_used_nodes.append(node[0])

    # 确定节点的层级（最终确定）
    nodes_tuples_fixed_list_final = []

    for node in all_used_nodes:
        if node in nodes_inputs:
            nodes_tuples_fixed_list_final.append([node, 0])
        elif node in nodes_outputs:
            nodes_tuples_fixed_list_final.append([node, max_length - 1])
        else:
            list_node = []
            for nodes in nodes_tuples_fixed_list:
                if node == nodes[0]:
                    list_node.append(nodes)
            layer = 0
            for ele in list_node:
                # 删除重复项，取最大
                if ele[1] > layer:
                    layer = ele[1]
            nodes_tuples_fixed_list_final.append([node,layer])


    # 每个层级的节点合数
    length_of_layers = []
    number_of_nodes = 0
    for index in range(max_length):
        for nodes in nodes_tuples_fixed_list_final:
            node, node_index = nodes[0], nodes[1]
            if node_index == index: 
                number_of_nodes += 1
        length_of_layers.append(number_of_nodes)

    length_of_each_layer = []
    length_of_each_layer.append(length_of_layers[0])
    for i in range(1, max_length):
        length_of_each_layer.append(length_of_layers[i] - length_of_layers[i-1])

    # 输出每个层级的节点个数
    #print("length of each layers:", length_of_each_layer)

    # 所有端到端的路
    all_path_side2side = []
    for path in all_path:
        if len(path) == 2:
            all_path_side2side.append(path)
        else:
            for i in range(len(path)-1):
                all_path_side2side.append([path[i],path[i+1]])

    # 定义节点到绘图的字典
    dict_nodes_graph = dict()
    count_number_layer = [0] * max_length
    for nodes in nodes_tuples_fixed_list_final:
        node, layer = nodes
        dict_nodes_graph[node] = [count_number_layer[layer], layer]
        count_number_layer[layer] += 1
    list_nodes_number.append(np.sum(length_of_each_layer))
    list_connection_number.append(len(all_path_side2side))
    #print("Number of nodes:{} Number of connections:{}".format(np.sum(length_of_each_layer),len(all_path_side2side)))
print("Total nodes:{} Total connections::{}".format(np.sum(list_nodes_number), np.sum(list_connection_number)))

Total nodes:3631 Total connections::4394


In [11]:
local_dir = os.getcwd()
config_path = os.path.join(local_dir, "config-feedforward-letter2")

config = neat.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    config_path,
)

def get_pred_real(i, j):
    winner_net = neat.nn.FeedForwardNetwork.create(winner_list[i], config)
    
    numbers = [*range(26)]
    class1 = classes[i]
    class2 = []
    class1_indexes = []
    class2_indexes = []
    
    test_X = testX[j]
    test_y = testy[j]

    list_y = [test_y]
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)


    testsamplesize = 1
    test_x_inputs =[tuple(test_X)]
    test_x_outputs = [tuple([c]) for c in class1_label + class2_label]

    outputs = []
    for xi in test_x_inputs:
        output = winner_net.activate(xi)
        outputs.append(output)

    px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 2), axis=1)
    
    # the index of maximum in each line
    return px_outputs

from random import randint
pred_value = []
error_list = []

for j in range(400):
    pred = []
    for i in range(necoc):
        pred_outputs = get_pred_real(i, j)
        pred.append(pred_outputs)

    error = []
    for i in range(26):
        loss = 0
        for ii in range(len(pred)):
            #print(number_codes[0][ii], pred[ii][0])
            #print (pred[ii][0][number_codes[0][ii]])
            loss += (1 - pred[ii][0][number_codes[i][ii]])**2
        error.append(loss)

    pred_value.append(np.where(error==np.min(error)))

list_P = []
for i in pred_value:
    if (len(i[0])) == 1:
        list_P.append(i[0][0])
    else:
        random_pick = randint(0, len(i[0])-1)
        list_P.append(i[0][random_pick])
np.sum(list_P == testy)/400

0.34